<a href="https://colab.research.google.com/github/jpantojaj/Backtesting_Stresstesting/blob/main/Sesi%C3%B3n_8_Modelos_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importamos librerias y data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [ ]:
raw_csv_data = pd.read_csv("flujo_caja.csv")

## Exploramos la data

In [ ]:
raw_csv_data

In [ ]:
raw_csv_data.sort_values(by = ['Date'])

In [ ]:
raw_csv_data.info()

In [ ]:
raw_csv_data['Date'] = pd.to_datetime(raw_csv_data['Date'])
raw_csv_data.info()

In [ ]:
raw_csv_data

In [ ]:
raw_csv_data.set_index('Date', inplace=True)

In [ ]:
raw_csv_data

In [ ]:
raw_csv_data.info()

In [ ]:
raw_csv_data.isnull().sum()

In [ ]:
raw_csv_data.describe()

In [ ]:
raw_csv_data.plot(figsize = (15,7), linewidth = 3, title = 'FlujoNetoRelativizado')
plt.grid()
plt.show()

In [ ]:
px.line(raw_csv_data, title='FlujoNetoRelativizado')

In [ ]:
std_daily = raw_csv_data['FNR'].std()
print('Volatilidad Diaria: ', '{:.2f}%'.format(std_daily))

In [ ]:
std_monthly = sqrt(21) * std_daily
print ('Volatilidad Mensual: ', '{:.2f}%'.format(std_monthly))

In [ ]:
std_annual = sqrt(252) * std_daily
print ('Volatilidad Anual: ', '{:.2f}%'.format(std_annual))

## Generamos estimaciones para la volatilidad

In [ ]:
size = int(len(raw_csv_data)*0.8)
df, df_test = raw_csv_data.iloc[:size], raw_csv_data.iloc[size:]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
# GARCH (Supuesto Normal)
basic_gm = arch_model(raw_csv_data['FNR'], p = 1, q = 1,
                      mean = 'constant', vol = 'GARCH', dist = 'normal')
gm_result = basic_gm.fit()

In [ ]:
gm_result.summary()

In [ ]:
gm_result.plot()
plt.show()

In [ ]:
gm_forecast = gm_result.forecast(horizon = 5)
print(gm_forecast.variance[-1:])

In [ ]:
gm_result2 = basic_gm.fit(last_obs='2017-11-29', update_freq = 10)
pred_gm = gm_result2.forecast(horizon = 1, align='target')

In [ ]:
pred_gm.variance

In [ ]:
pred_gm.mean

In [ ]:
plt.title("Predicciones de Volatilidad", size = 24)
plt.plot(pred_gm.variance['2017-11-29':], color = 'red')
plt.plot(df_test.FNR, color = 'blue')
plt.show()

In [ ]:
# Obtiene los residuales de estimación y la volatilidad
gm_resid = gm_result.resid
gm_std = gm_result.conditional_volatility

# calcula los residuales estandarizados
gm_std_resid = gm_resid /gm_std

In [ ]:
# Histograma de Residuales estandarizados
plt.figure(figsize=(10, 5))
plt.hist(gm_std_resid, bins = 50,
         facecolor = 'orange', label = 'Standardized residuals')
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(gm_std_resid, alpha = 0.05, zero = False)
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test_gm = acorr_ljungbox(gm_std_resid , lags = 10)
print('P-values are: ', lb_test_gm[1])

In [ ]:
from scipy.stats import shapiro
shapiro_results = shapiro(gm_std_resid)
print("Shapiro results:", shapiro_results)
p_value = shapiro_results[1]
print("P-value: ", p_value)

In [ ]:
# GARCH (Supuesto t)
skewt_gm = arch_model(raw_csv_data['FNR'], p = 1, q = 1, mean = 'constant', vol = 'GARCH', dist = 'skewt')
skewt_result = skewt_gm.fit()

In [ ]:
skewt_result.summary()

In [ ]:
skewt_result.plot()
plt.show()

In [ ]:
skewt_forecast = skewt_result.forecast(horizon = 5)
print(skewt_forecast.variance[-1:])

In [ ]:
skewt_result2 = skewt_gm.fit(last_obs='2017-11-29', update_freq = 10)
pred_skewt_gm = skewt_result2.forecast(horizon = 1, align='target')
pred_skewt_gm

In [ ]:
plt.title("Predicciones de Volatilidad", size = 24)
plt.plot(pred_skewt_gm.variance['2017-11-29':], color = 'red')
plt.plot(df_test.FNR, color = 'blue')
plt.show()

In [ ]:
# Obtiene los residuales de estimación y la volatilidad
skm_resid = skewt_result.resid
skm_std = skewt_result.conditional_volatility

# calcula los residuales estandarizados
skm_std_resid = skm_resid /skm_std

In [ ]:
# Histograma de Residuales estandarizados
plt.figure(figsize=(10, 5))
plt.hist(skm_std_resid, bins = 50,
         facecolor = 'orange', label = 'Standardized residuals')
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(skm_std_resid, alpha = 0.05, zero = False)
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test_skm = acorr_ljungbox(skm_std_resid , lags = 10)
print('P-values are: ', lb_test_skm[1])

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(skm_std, color = 'gold', label = 'Skewed-t Volatility')
plt.plot(gm_std, color = 'red', label = 'Normal Volatility')
plt.plot(raw_csv_data['FNR'], color = 'grey',
         label = 'FNR', alpha = 0.4)
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
#hallaremos diferentes aproximaciones cambiando la media para realizar comparaciones
zeromean_gm = arch_model(df['FNR'], p = 1, q = 1, mean = 'zero', vol = 'GARCH')
zmean_result = zeromean_gm.fit()

cmean_gm = arch_model(df['FNR'], p = 1, q = 1, mean = 'constant', vol = 'GARCH')
cmean_result = cmean_gm.fit()

armean_gm = arch_model(df['FNR'], p = 1, q = 1, mean = 'AR', lags=1, vol = 'GARCH')
armean_result = armean_gm.fit()

In [ ]:
print(zmean_result.summary())
print(cmean_result.summary())
print(armean_result.summary())

In [ ]:
zmean_vol=zmean_result.conditional_volatility
cmean_vol=cmean_result.conditional_volatility
armean_vol=armean_result.conditional_volatility

In [ ]:
zmean_vol.head()

In [ ]:
cmean_vol.head()

In [ ]:
armean_vol.head()

In [ ]:
# Plot model volatility
plt.plot(zmean_vol, color = 'green', label = 'Zero Mean Volatility')
plt.plot(cmean_vol, color = 'blue', label = 'Constant Mean Volatility')
plt.plot(armean_vol, color = 'red', label = 'AR Mean Volatility')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
print(np.corrcoef(zmean_vol[1:], cmean_vol[1:])[0,1])
print(np.corrcoef(cmean_vol[1:], armean_vol[1:])[0,1])

## Calcular el VaR

In [ ]:
# Quantile Empírico
q_empirical = gm_std.quantile(0.05)
print('5% Empirico: ', q_empirical)

In [ ]:
# Calcula el VaR Empírico
VaR_empirical = pred_gm.mean.values + np.sqrt(pred_gm.variance).values * q_empirical
VaR_empirical = pd.DataFrame(VaR_empirical, columns = ['5%'], index = pred_gm.variance.index)
VaR_empirical

In [ ]:
# Grafica el VaR
plt.plot(VaR_empirical, color = 'brown', label = '5% Empirical VaR')
plt.scatter(pred_gm.variance.index,raw_csv_data.FNR, color = 'orange', label = 'FNR' )
plt.legend(loc = 'upper right')
plt.show()